In [ ]:
%pip install tensorflow opencv-python matplotlib kagglehub

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shreelakshmigp/cedardataset")

print("Path to dataset files:", path)

In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

Load Data

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(os.path.join(path, "signatures"), batch_size=256)

Pre Processing

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
scaled_iterator = data.as_numpy_iterator()

In [ ]:
batch = scaled_iterator.next()

In [ ]:
batch[1].max()

In [ ]:
fig, ax = plt.subplots(ncols = 4, figsize = (20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])
    print(batch)

Test Train Split

In [ ]:
len(data)

In [ ]:
train_size = int(0.6*len(data))
test_size = int(0.3*len(data))
val_size = int(0.1*len(data))+1
print(train_size, test_size, val_size)

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

Building the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [ ]:
model.summary()

Train

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')

In [ ]:
hist = model.fit(train, epochs = 25, validation_data = val, callbacks=[tensorboard_callback])

In [ ]:
hist.history

Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='blue', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.show

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='blue', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.show()

Performance Evaluation

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
precision = Precision()
recall = Recall()
Baccuracy = BinaryAccuracy()

In [ ]:
len(test)

In [ ]:
for batch in test.as_numpy_iterator():
    X,y = batch
    yhat = model.predict(X)
    precision.update_state(y, yhat)
    recall.update_state(y, yhat)
    Baccuracy.update_state(y, yhat)

In [ ]:
print("Precision: ", precision.result().numpy())
print("Recall: ", recall.result().numpy())
print("Accuracy: ", Baccuracy.result().numpy())

Saving model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models', 'forgeryclassifier.keras'))

In [ ]:
loaded_model = load_model(os.path.join('models', 'forgeryclassifier.keras'))

In [ ]:
forged_example = cv2.imread('forgeries_1_14.png')
forged_example = tf.image.resize(forged_example, (256, 256))
loaded_model.predict(np.expand_dims(forged_example/255, 0))